# Code to convert from Andy's "wstd" LC files to SNANA format

Python 2.7

## Notes

- Input files:
    - Wstd files
    - metadata file with (snname, zhelio, e_zhel, zcmb, e_zcmb, sample, ra, dec)
    - metadata file with (snname, TBmax, err_TBmax, EBV_MW, err_EBV_MW)
    
- The output files will be saved in a subfolder called 'snana' 

In [1]:
import numpy as np
import glob # To read the files in my directory
import os # To use command line like instructions
import datetime # Get the current date and time

cc = 299792.458  # Speed of light (km/s)

#--------------------------------------------------------60
code_created_by = 'Arturo_Avelino'
# On date: 2017.01.10 (yyyy.mm.dd)
code_name = '03_WstdAndy_to_SNANA.ipynb'
code_version = '0.1.9'
code_last_update = '2019.07.03'
code_location = 'https://github.com/ArturoAvelino/SALT2fit/blob/master/03_WstdAndy_to_SNANA.py'

In [2]:
##############################################################################80

# User

In [3]:
# Directory where Andy's wstd files to be converted are located:
dirwstd = '/Users/arturo/Documents/Research/Workplace/BayeSED/andy_comp/Wstd_to_SNANA/Wstd/'

# List of SNe filenames to be converted.
file_listSNe = 'List_SNe_Notes_.txt'

# Location of the list file:
dirfilelist = '/Users/arturo/Documents/Research/Workplace/BayeSED/andy_comp/NoShared/'

#--------------------------------------------------------60

# String to be printed in the 'SURVEY' line in SNANA files.
# NOTE: For low-z CSP data set "Survey = 'CSP' ". This allows to fit the data in 
# SNANA/SALT2 with no issues when using the CSP kcorr file in SNANA.
# Options:
#      'LOWZ' for low-z CfA and Others LCs.
#      'CSP' for low-z CSP LCs.
Survey = 'LOWZ'  

#--------------------------------------------------------60

# Directory to save the output:
## DirSaveOutput = dirwstd+'snana/'
DirSaveOutput = '/Users/arturo/Documents/Research/Workplace/BayeSED/andy_comp/Wstd_to_SNANA/snana/'

#### Filter's name matching between Andy and SNANA

In [4]:
# Filter's name matching between Andy and SNANA

# Write -all- the filter's name in the line "FILTERS: " in the 
# SNANA-format LC file? If 'False', then it will be written the 
# filters listed in the list 'filterListToConvert' below only. If 
# 'True', then it will write down -all- the filters names. For those
# that are not in the list 'filterListToConvert' then it will
# write the filter's name using Andy's original names.
write_all_filters = True 

# List of filter names that will be converted their names from Andy's 
# to SNANA convention's names.
# Any other filter not listed here will not be written in the
# line "FILTERS: " in the SNANA-like output file, 
# unless "write_all_filters = True".

# filterListToConvert = ['r_prime', 'i_prime', 
#                     'B_CTIO1p3m','V_CTIO1p3m','R_CTIO1p3m','I_CTIO1p3m',
#                     'u_CSP', 'g_CSP', 'r_CSP', 'i_CSP', 'B_CSP', 'V_CSP',
#                     'B', 'V', 'R', 'I']

filterListToConvert = ['null']

#--------------------------------------------------------60
# Create a dictionary with the conversion names from the above list.

# The dictionary structure is: (filter_Andy: filter_SNANA)
FilterNameConversion_dict = {}

FilterNameConversion_dict['null'] = ['null']

# FilterNameConversion_dict['r_prime'] = ['r']
# FilterNameConversion_dict['i_prime'] = ['i']
# FilterNameConversion_dict['B_CTIO1p3m'] = ['B'] 
# FilterNameConversion_dict['V_CTIO1p3m'] = ['V'] 
# FilterNameConversion_dict['R_CTIO1p3m'] = ['R'] 
# FilterNameConversion_dict['I_CTIO1p3m'] = ['I'] 
# FilterNameConversion_dict['u_CSP'] = ['u']
# FilterNameConversion_dict['g_CSP'] = ['g']
# FilterNameConversion_dict['r_CSP'] = ['r']
# FilterNameConversion_dict['i_CSP'] = ['i']
# FilterNameConversion_dict['B_CSP'] = ['B']
# FilterNameConversion_dict['V_CSP'] = ['o']
# 
# FilterNameConversion_dict['B'] = ['B'] 
# FilterNameConversion_dict['V'] = ['V'] 
# FilterNameConversion_dict['R'] = ['R'] 
# FilterNameConversion_dict['I'] = ['I'] 

#--------------------------------------------------------60
# Rename the output files using the first 'TrimFileName' characters 
# of the datafile.

# "-4" = use the full name of the file except the extension characters ".dat"
TrimFileName = -4 
# TrimFileName = 18 

# Print on the output files the date-time and script used to 
# create them?
print_date_scriptName = True 

In [5]:
##############################################################################80

-------

# Automatic

In [6]:
#- Force the creation of the directory to save the outputs.
#- "If the subdirectory does not exist then create it"
if not os.path.exists(DirSaveOutput): os.makedirs(DirSaveOutput)

#### Function to identify string or number

In [7]:
# Function to identify if a string is an integer number or a letter.
# This will be used in the dictionary construction to properly read some SN names.

def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

# Tests
print '#', is_number('5'), is_number('e')
# True False

# True False


In [8]:
# Get the current date and time
# Read the time and date now
now = datetime.datetime.now()

## Input metadata

### Read metadata (snname, zhelio, e_zhel, zcmb, e_zcmb, sample, ra, dec)

In [9]:
MetadataFile = 'carrick_Flow_corrections_snnames_v1.txt'
DirMetadata = '/Users/arturo/Documents/Research/Workplace/BayeSED/andy_comp/NoShared/'

# Reading the metadata file
infoSNe_data = np.genfromtxt(DirMetadata+MetadataFile,
                            dtype=['S17', float,float, 'S40',float,float,
                                   float,float,float,float,'S16', int ])

# Create a dictionary:
# {snname: zhelio, e_zhel, zcmb, e_zcmb, zcmbFlow, 
#          e_zcmbFlow, code, ra, dec}

InfoSN_dict = {infoSNe_data['f0'][i]: [
                infoSNe_data['f4'][i]/cc, infoSNe_data['f5'][i]/cc,
                infoSNe_data['f6'][i]/cc, infoSNe_data['f7'][i]/cc,
                infoSNe_data['f8'][i]/cc, infoSNe_data['f9'][i]/cc,
                infoSNe_data['f11'][i],
                infoSNe_data['f1'][i], infoSNe_data['f2'][i] ]
                for i in range(len(infoSNe_data)) }


#--------------------------------------------------------60
# Just check that the dictionary works fine.

InfoSN_dict['sn1998bu']
# [0.0029620491653595902,
#  3.3356409519815205e-06,
#  0.0023683050759068795,
#  8.6726664751519533e-05,
#  0.003138838135814611,
#  0.00050034614279722807,
#  1,
#  161.69166999999999,
#  11.835279999999999]

[0.00296204916535959,
 3.3356409519815205e-06,
 0.0025250802006500112,
 0.00023349486663870643,
 0.000807225110379528,
 0.0005003461427972281,
 1,
 161.69167,
 11.83528]

In [10]:
#-----------------------------------------------------------------------------80

### Read metadata (t_Bmax, EBV_MW)

In [11]:
MetadataFile2 = 'tBmax_EBVMW_all.txt'
DirMetadata2 = '/Users/arturo/Documents/Research/Workplace/BayeSED/andy_comp/NoShared/'

DistMu_np = np.genfromtxt(DirMetadata2+ MetadataFile2,
                             dtype=['S16',float,float,float,float])

#----- Create a dictionary -----
# (snname: TBmax, err_TBmax, EBV_MW, err_EBV_MW)

DistMu_dict ={}
for i in range(len(DistMu_np)):
    
    # Sn name
    snName_1 = DistMu_np['f0'][i]
    
    TBmax_int_1 = DistMu_np['f1'][i]
    
    # add 53000 MJD to tBmax of some CSP SNe.
    if TBmax_int_1 < 4000:
        TBmax_int = TBmax_int_1 + 53000
    else: TBmax_int = TBmax_int_1
    
    err_TBmax_int = DistMu_np['f2'][i]
    
    EBV_MW_int     = DistMu_np['f3'][i]
    err_EBV_MW_int = DistMu_np['f4'][i]
    
    DistMu_dict[snName_1] = [TBmax_int, err_TBmax_int, EBV_MW_int, err_EBV_MW_int]

#--------------------------------------------------------60
# Just check that the dictionary works fine.

print '#', DistMu_dict['sn1998bu']
# # [50953.113987516, 0.0811452888, 0.0217, 0.0002]

# [50953.113987516, 0.0811452888, 0.0217, 0.0002]


#### Convert fluxes to zeropoint = 27.5 (default in SNANA)

In [12]:
zp_snana = 27.5

def flux_snana(flux_old, zp_Andy): 
    
    zp_snana = 27.5
    flux_new = flux_old * 10**(0.4*(zp_snana - zp_Andy))
    
    return flux_new

print '# Test:', flux_snana(587.41, 25)
# Test: 5874.1

# Test: 5874.1


In [13]:
#-----------------------------------------------------------------------------80

-----

# Read/convert Andy's wstd file to SNANA-format text file

In [14]:
# Read the names of all the photometry files to be converted

the_list_np = np.genfromtxt(dirfilelist+file_listSNe, dtype=['S170'])

# Convert the numpy array to list:
the_list = the_list_np['f0'].tolist()

print '# %s SNe in this list.'%len(the_list)
# 341 SNe in this list.

# 341 SNe in this list.


In [15]:
##############################################################################80

### Main loop

In [16]:
zp_Andy = 25  # Andy's zeropoint: 25 mag 

count_lcs = 0 # count LCs converted to SNANA format

# Loop over Andy's wstd LC files.
for wstdFile in the_list:
    
    wstd_np = np.genfromtxt(dirwstd+wstdFile, usecols=[1,2,3,4,5] , 
                        dtype=[float,'S14', float, float, float])
    
    #====================================================
    
    # Create the variable "snName" containing the first 8 (or 7) 
    # letters of the SNe file name
    try:
        if   wstdFile[7] == '_': 
            snName = wstdFile[:7]  # To read correctly, e.g., "sn2011B_"
        elif wstdFile[7] != '_':
            # To read correctly, e.g., "snf20080514-002"
            if is_number(wstdFile[7]): snName = wstdFile[:15] 
            else: snName = wstdFile[:8]  # To read correctly, e.g., "sn1998bu"  
    except: snName = wstdFile[:6]  # To read correctly, e.g., "sn2011B"

    #-------------------------------
    # Create a list of filters that are in a given photometric file:

    # Reset
    ListFilters = []
    filtername_int_2 = ''; filtersnana_2 = '';
    
    # Loop over the photometry:
    for j in range(len(wstd_np)):

        # Read the filter's name
        filtername_int_2 = wstd_np['f1'][j]

        # Convert the filter's name to SNANA
        if filtername_int_2 in filterListToConvert:
            filtersnana_2 = FilterNameConversion_dict[filtername_int_2][0]
        else:
            if write_all_filters:
                filtersnana_2 = filtername_int_2

        # Create a list of unique filters 
        if filtersnana_2 not in ListFilters:
            ListFilters += [filtersnana_2]

    # print '# Filters in this file: ',ListFilters

    #---- Create a single string with the name of all the filters ----
    # This will be written in the SNANA-format text file 
    # in the row "FILTERS: "

    ListFiltersToPrint = ''
    for name in ListFilters:
        ListFiltersToPrint = ListFiltersToPrint+name

    # print '# Text to print in the field FILTERS:', ListFiltersToPrint
    
    #--------------------------------------
    # Determine the number of observations based on the filters
    # to be considered
    
    NOBS = 0
    
    # Loop over the photometry:
    for j2 in range(len(wstd_np)):
        # Read the filter's name
        filtername_int_2 = wstd_np['f1'][j2]
        
        if write_all_filters:
            NOBS += 1
        elif filtername_int_2 in filterListToConvert:
            NOBS += 1
    
    #====================================================
    
    # Read the time and date right now
    now = datetime.datetime.now()

    snana_file = open(DirSaveOutput+wstdFile[:TrimFileName]+'_snana.dat', 'w')

    text_line_1 = '#'+'-'*60+'\n'
    snana_file.write('#      %s \n'%snName)
    snana_file.write("# Andy Friedman's Wstd file converted to SNANA-like format. \n")
    snana_file.write('# Source file: %s \n'%wstdFile)
    snana_file.write("# Metadata information from: %s \n"%MetadataFile)
    snana_file.write("# Photometry and metadata compiled by Andrew S. Friedman (UCSD).\n")
    # snana_file.write("# Located at: \n")
    # snana_file.write("# %s \n"%DirMetadata)
    snana_file.write("# write_all_filters = %s. \n"%write_all_filters)
    snana_file.write("# Zeropoint of the converted SNANA file: %s mag\n"%zp_snana)
    
    snana_file.write(text_line_1)
    snana_file.write('# Data table created by: %s\n'%code_created_by)
    if print_date_scriptName: 
        text_01 = now.strftime("%Y.%m.%d (yyyy.mm.dd); %H:%M hrs (ET).")
        snana_file.write('# On date: %s \n'%text_01)
        snana_file.write('# Script used: %s \n'%code_name)
        snana_file.write('# Script version: %s \n'%code_version)
        # snana_file.write('# Script URL location:\n')
        # snana_file.write('# %s\n'%code_location)
    snana_file.write(text_line_1)

    #---------------------

    #   SNANA header

    # Create a final dictionary. It has the form of:
    # (snname: zhelio, err_zhelio, zcmb, err_zcmb) 
    
    zhel = InfoSN_dict[snName][0]
    err_zhel = InfoSN_dict[snName][1]
    
    # Flag to determine the appropiate z_cmb:
    flag_zcmb  = InfoSN_dict[snName][6]
    if flag_zcmb > 0.1:
        zcmb     = InfoSN_dict[snName][2]
        err_zcmb = InfoSN_dict[snName][3]
    else:
        zcmb     = InfoSN_dict[snName][4]
        err_zcmb = InfoSN_dict[snName][5]

    RA       = InfoSN_dict[snName][7]
    DEC      = InfoSN_dict[snName][8]
    
    EBV_MW  = DistMu_dict[snName][2]
    PEAKMJD = DistMu_dict[snName][0]

    snana_file.write('SURVEY: %s \n'%Survey)
    snana_file.write('SNID: %s \n'%snName[2:])
    snana_file.write('IAUC: %s \n'%snName[2:])
    snana_file.write('RA: %s deg \n'%RA)
    snana_file.write('DECL: %s deg \n'%DEC)
    snana_file.write('MWEBV: %s MW E(B-V) \n'%EBV_MW)
    snana_file.write('REDSHIFT_HELIO: %1.5f +- %1.6f (HEL)\n'%(zhel, err_zhel))
    snana_file.write('REDSHIFT_CMB: %1.5f +- %1.6f (CMB)\n'%(zcmb, err_zcmb))
    snana_file.write('REDSHIFT_FINAL: %1.5f +- %1.6f (CMB)\n'%(zcmb, err_zcmb))
    snana_file.write('SEARCH_PEAKMJD: %.3f \n'%PEAKMJD)
    snana_file.write('FILTERS: %s \n'%ListFiltersToPrint)

    #---------------------

    snana_file.write(text_line_1)
    # old. snana_file.write('NOBS: %s \n'%len(wstd_np))
    snana_file.write('NOBS: %s \n'%NOBS)
    snana_file.write('NVAR: 7 \n')
    snana_file.write('VARLIST:  MJD  FLT           FIELD       \
FLUXCAL         FLUXCALERR  MAG  MAGERR  \n')

    #---------------------

    # Loop over photometry:
    for i in range(len(wstd_np)):

        filtername_int1 = wstd_np['f1'][i]

        if filtername_int1 in filterListToConvert:
            filtersnana = FilterNameConversion_dict[filtername_int1][0]
        else: filtersnana = filtername_int1

        flux_old         = wstd_np['f2'][i]
        err_flux_old_low = wstd_np['f3'][i]
        err_flux_old_hig = wstd_np['f4'][i]

        flux_new         = flux_snana(flux_old, zp_Andy)
        err_flux_new_low = flux_snana(err_flux_old_low, zp_Andy)
        err_flux_new_hig = flux_snana(err_flux_old_hig, zp_Andy)

        average_errorFlux = (err_flux_new_low + err_flux_new_hig)/2

        # Write the line in the text file
        if write_all_filters:
            snana_file.write('OBS: %.3f  %-12s NULL  %15.4f \
%15.4f    0     0 \n'%(wstd_np['f0'][i], 
                        filtersnana, flux_new, average_errorFlux))
            
        elif filtername_int1 in filterListToConvert:
            snana_file.write('OBS: %.3f  %-12s NULL  %15.4f \
%15.4f    0     0 \n'%(wstd_np['f0'][i], 
                        filtersnana, flux_new, average_errorFlux))

    snana_file.write('END:')
    snana_file.close();

    count_lcs += 1
    print wstdFile[0:40]
    
#--------------------------------------------------------60
print '\n# All %s LCs converted smoothly'%count_lcs

sn1998bu__U_69_B_97_V_110_R_87_I_108_J_1
sn1999aa__B_19_V_18_R_16_I_16__LOSS_gane
sn1999aa__U_21_B_23_V_21_R_21_I_22__CfA2
sn1999aa__U_24_B_58_V_57_R_53_I_53_J_4_K
sn1999cl__B_22_V_22_R_21_I_21__LOSS_gane
sn1999cl__U_8_B_7_V_9_R_8_I_8__CfA2_jha0
sn1999cl__U_8_B_15_V_17_R_16_I_16_J_6_H_
sn1999cp__B_5_V_5_R_5_I_5_J_2_H_2_K_2__L
sn1999cp__B_27_V_29_R_28_I_27__LOSS_gane
sn1999ee__U_45_B_118_V_126_R_53_I_109_J_
sn1999ek__U_2_B_5_V_7_R_6_I_6__CfA2_jha0
sn1999ek__U_2_B_38_V_39_R_37_I_37_J_14_H
sn1999gp__B_23_V_23_R_21_I_20__LOSS_gane
sn1999gp__U_11_B_11_V_11_R_10_I_11__CfA2
sn1999gp__U_11_B_47_V_49_R_44_I_39_J_3_H
sn2000bh__B_12_V_12_R_9_I_11_Y_CSP_6_J_C
sn2000bk__B_9_V_9_R_9_I_9_J_18_H_16__LCO
sn2000ca__U_6_B_18_V_18_R_18_I_18_J_CSP_
sn2000ce__U_6_B_6_V_6_R_6_I_6__CfA2_jha0
sn2000ce__U_6_B_13_V_13_R_13_I_13_J_3_H_
sn2000E__U_5_B_27_V_26_R_27_I_9_J_16_H_1
sn2001ba__B_30_V_34_I_33_J_CSP_15_H_CSP_
sn2001bt__B_22_V_23_R_20_I_22_J_21_H_21_
sn2001cn__U_3_B_25_V_25_R_25_I_25_J_19_H
sn2001cz__B_14_V

In [17]:
snana_file.close();snana_file.close();snana_file.close();

----